In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from mirage import FolderRawStorage, WhooshChunkStorage, WordCountingChunkingAlgorithm
from loguru import logger
raw_doc = FolderRawStorage(folder_path='data_txt')
chunks = WhooshChunkStorage(scoring_function='BM25F', normalizer=True)
chunking_algorithm = WordCountingChunkingAlgorithm(
    raw_storage=raw_doc,
    chunk_storage=chunks,
    words_amount=128
)
chunking_algorithm.execute()

2025-04-27 19:01:19.802 | INFO     | mirage.index.chunking_algorithms.WordCountingChunkingAlgorithm:chunk_a_document:28 - Reading a document... Приказ Росавиации от 28.12.2022 N 970-П  Об утверждении Норм.txt
2025-04-27 19:01:22.469 | INFO     | mirage.index.chunking_algorithms.WordCountingChunkingAlgorithm:chunk_a_document:28 - Reading a document... Приказ_Минтранса_РФ_от_21_11_2005_N_139_ред_от_17_09_2010.rtf.txt
2025-04-27 19:01:23.105 | INFO     | mirage.index.chunking_algorithms.WordCountingChunkingAlgorithm:chunk_a_document:28 - Reading a document... ФАП-10.txt
2025-04-27 19:01:23.986 | INFO     | mirage.index.chunking_algorithms.WordCountingChunkingAlgorithm:chunk_a_document:28 - Reading a document... ФАП-109.txt
2025-04-27 19:01:24.247 | INFO     | mirage.index.chunking_algorithms.WordCountingChunkingAlgorithm:chunk_a_document:28 - Reading a document... ФАП-128.txt
2025-04-27 19:01:27.272 | INFO     | mirage.index.chunking_algorithms.WordCountingChunkingAlgorithm:chunk_a_docume

344

In [9]:
from mirage.embedders import HuggingFaceEmbedder
from mirage.index.vector_index.FaissVectorIndex import FaissIndexFlatL2, FaissIndexFlatIP


emb = HuggingFaceEmbedder(model_name='BAAI/bge-m3')
vectors = FaissIndexFlatIP(dimensionality=emb.get_dimensionality())
emb.convert_chunks_to_vector_index(
        chunk_storage=chunks,
        vector_index=vectors, visualize=True
)

2025-04-27 21:53:30.855 | DEBUG    | mirage.embedders.HuggingFaceEmbedder:__init__:12 - cpu
2025-04-27 21:53:36.648 | INFO     | mirage.embedders.HuggingFaceEmbedder:convert_chunks_to_vector_index:4 - 313
  0%|          | 0/313 [00:01<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from mirage.index import MirageIndex


index = MirageIndex(
    raw_storage=raw_doc, chunk_storage=chunks, chunking_algorithm=chunking_algorithm, vector_index=vectors
)


In [ ]:
index.save('main.mirage_index')

2025-04-27 16:13:38.641 | INFO     | mirage.index.MirageIndex:save:32 - Saving Mirage index to main.mirage_index...
2025-04-27 16:13:39.345 | INFO     | mirage.index.MirageIndex:save:78 - Mirage index saved to main.mirage_index.


In [ ]:
index_loaded = MirageIndex.load('main.mirage_index')

2025-04-27 16:13:41.432 | INFO     | mirage.index.MirageIndex:load:84 - Loading Mirage index from main.mirage_index...
2025-04-27 16:13:41.593 | INFO     | mirage.index.MirageIndex:load:127 - Mirage index loaded from main.mirage_index.


In [ ]:
results = index_loaded.vector_index.query(
    query_vector=emb.embed('убийство'), top_k=10
)
index_loaded.chunk_storage.get_texts_for_search_results(results)

['рвут на себе волосы,\xa0— сказал Холмс вечером, когда мы обсуждали это событие.\xa0— Он умер, и пропали все их надежды на шумную рекламу.\r\n\r\n—\xa0По-моему, они мало что сделали для поимки преступника,\xa0— заметил я.\r\n\r\n—\xa0В этом мире неважно, сколько вы сделали,\xa0— с горечью произнес Холмс.\xa0— Самое главное — суметь убедить людей, что вы сделали много. Но все равно,\xa0— продолжал он после паузы, уже веселее,\xa0— я ни за что не отказался бы от этого расследования. Я не помню более интересного дела. И как оно ни просто, все же в нем было немало поучительного.\r\n\r\n—\xa0Просто?!\xa0— воскликнул я.\r\n\r\nХолмса рассмешило мое изумление.\r\n\r\n—\xa0Разумеется, его никак нельзя назвать сложным,\xa0— сказал он.\xa0— И вот вам доказательство — за три дня я без всякой помощи и только посредством самых обыкновенных умозаключений сумел установить личность преступника.\r\n\r\n—\xa0Это верно!\r\n\r\n—\xa0Я уже как-то говорил вам, что необычное — скорее помощь, чем помеха в на

In [27]:
from math import ceil


doctext = raw_doc['ФАП-32.txt']
split = doctext.split()
SIZE = 1000
doctext = [
    ' '.join(split[i * SIZE: (i + 1) * SIZE]) 
    for i in range(ceil(len(split) / SIZE))
]

len(doctext)

8

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    'BAAI/bge-m3', use_fast=True
)

In [31]:
from pprint import pprint


encoding = tokenizer(doctext[0], return_offsets_mapping=True)

print(encoding)


{'input_ids': [0, 6, 146306, 31078, 47687, 73968, 1986, 86413, 177187, 1709, 6, 203632, 1560, 65837, 1560, 213866, 4560, 68903, 161010, 46350, 4318, 15, 146306, 154456, 40048, 6, 203632, 1560, 86042, 16, 2429, 5729, 804, 1382, 536, 1522, 5188, 120035, 49108, 45662, 85747, 121210, 22621, 44, 4754, 198531, 1447, 68146, 4, 227554, 77472, 718, 132094, 4256, 35, 53224, 94490, 58, 6, 82626, 241524, 161972, 145797, 312, 85747, 8441, 167, 417, 20960, 135, 225098, 50370, 183, 953, 35185, 10586, 1070, 5, 438, 42698, 81518, 44, 4754, 2354, 8899, 35141, 2192, 49869, 16753, 49108, 59, 22959, 2740, 15, 36627, 79094, 103, 6, 105087, 16753, 16286, 4, 10586, 4, 438, 427, 4, 6716, 5, 702, 17168, 74, 8272, 438, 29835, 6716, 5, 138, 161382, 74, 86097, 438, 2273, 4, 58, 44, 2225, 5, 18139, 966, 4, 438, 2678, 4, 6716, 5, 201, 128919, 74, 73584, 438, 17480, 6716, 5, 209, 19051, 74, 65942, 438, 496, 4, 6716, 5, 2789, 5039, 4, 2789, 12183, 74, 56434, 438, 106, 58, 44, 132, 1068, 5, 4879, 4, 6716, 5, 30777, 438

In [ ]:
emb.model.

TypeError: SentenceTransformer.forward() missing 1 required positional argument: 'input'

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3", use_fast=True)
model = AutoModel.from_pretrained("BAAI/bge-m3", trust_remote_code=True)

# Токенизация
inputs = tokenizer(doctext[0], return_tensors="pt", padding=True, truncation=True)

# Получение эмбеддингов
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state[:, 0]

In [36]:
embeddings

tensor([[-1.7280,  0.1247, -0.8666,  ..., -0.6453, -1.2181, -0.2229]])

2025-04-27 21:47:22.708 | INFO     | mirage.embedders.HuggingFaceEmbedder:convert_chunks_to_vector_index:4 - 2378
  0%|          | 0/2378 [00:00<?, ?it/s]


AttributeError: 'HuggingFaceEmbedder' object has no attribute 'tokenizer'

In [2]:
from mirage import MirageIndex

index = MirageIndex.load('indexes\W_128_0.1_intfloat_ip.mirage_index')

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\evgen\AppData\Local\Temp\ipykernel_17480\1343540182.py:3: SyntaxWarning: invalid escape sequence '\W'
  index = MirageIndex.load('indexes\W_128_0.1_intfloat_ip.mirage_index')
2025-04-27 23:45:00.145 | INFO     | mirage.index.MirageIndex:load:84 - Loading Mirage index from indexes\W_128_0.1_intfloat_ip.mirage_index...
2025-04-27 23:45:00.903 | INFO     | mirage.index.MirageIndex:load:127 - Mirage index loaded from indexes\W_128_0.1_intfloat_ip.mirage_index.


In [16]:
raw_storage_index = [
        index.chunk_storage.get_raw_index_of_document(qresult.chunk_storage_key)
        for qresult in index.chunk_storage.query('Предполетная подготовка')
    ][0]


In [17]:
index.raw_storage.get_link(raw_storage_index)

'data_txt_restored\\Приказ_Минтранса_РФ_от_21_11_2005_N_139_ред_от_17_09_2010.rtf.txt'